<a href="https://colab.research.google.com/github/alves-Moises/-programa-o_semana_py/blob/master/LGPD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1 Abrir cada arquivo de texto
from google.colab import files
import glob

arquivo = files.upload()
!unzip 'total.zip' -d 'total'

Saving total.zip to total (1).zip
Archive:  total.zip
replace total/total/lgpd64regular.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#2 Pré-processar os dados, implementando os seguintes procedimentos
from math import log
from collections import Counter
import itertools
import nltk
nltk.download('stopwords')
stops = nltk.corpus.stopwords.words('portuguese')
stops.remove('não')
# stops.remove('not')

nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('portuguese')


def tokenizar(str_texto):
    return word_tokenize(str_texto)

def limpar(lista):
    return [i.lower() for i in lista if i.isalpha()]

def sem_stops(lista):
    return [i for i in lista if i not in stops]

def stemizar(lista):
    return [stemmer.stem(i) for i in lista]

def achatar(lista):
    return list(itertools.chain(*lista))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import numpy as np
rng = np.random.RandomState(1)
textos = rng.randint(3, size=(4, 100))
classes = np.array([1, 2, 3, 4])

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(textos, classes)
MultinomialNB()
print(clf.predict(classes[2:3]))
[3]

ValueError: ignored

In [ ]:
#2
arqs = glob.glob('enron/*.txt')
mensagens = list()
for arq in arqs:
    arquivo = open(arq, 'r')
    classe = arquivo.readline()[0]  
    if classe == '1':
        classe = 'SPAM'
    else:
        classe = 'HAM'
    
    texto = arquivo.read()
    texto = stemizar(sem_stops(limpar(tokenizar(texto))))
    mensagens.append((texto, classe))
    arquivo.close()

treinamento = mensagens[:round(len(mensagens) * 0.8)]
teste = mensagens[len(treinamento) + 1:]

spam = [t[0] for t in treinamento if t[1] == 'SPAM']
ham = [t[0] for t in treinamento if t[1] == 'HAM']

In [ ]:
#3 Dividir o corpus pré-processado em treinamento (80%) e teste (20%).
from sklearn.model_selection import train_test_split
treinamento,teste = train_test_split(arqs,test_size=0.2, random_state = 0) 


NameError: ignored

In [ ]:
#4 Usando o corpus de treinamento, realizar as contagens do vocabulário, das classes (spam ou ham) e dos atributos (cada token corresponde a um atributo).
vocab1 = set(achatar(ham))|set(achatar(spam))

NameError: ignored

In [ ]:
len(vocab1)

33716

In [ ]:
n_docs_spam = len(spam)
n_docs_ham = len(ham)
n_docs_total = n_docs_spam + n_docs_ham

In [ ]:
n_vocab1 = len(vocab1)

In [ ]:
tokens_spam = achatar(spam)
cont_spam= Counter(tokens_spam)
tokens_ham = achatar(ham)
cont_ham = Counter(tokens_ham)

In [ ]:
n_tokens_spam = sum(cont_spam.values())
n_tokens_ham = sum(cont_ham.values())

In [ ]:
t=[i[0] for i in teste]

In [ ]:
teste_tokens = [w for w in teste]

In [ ]:
#5 Calcular as probabilidades relacionadas às contagens
def classifica(probabilidade_spam, probabilidade_ham):

    if probabilidade_spam > probabilidade_ham:

        return 'Email é Spam!'
    else:
        return 'Email não é Spam!'

In [ ]:
#6/7 Implementar uma função bayes() que receba uma mensagem e devolva a probabilidade de ela ser classificada como spam e como ham:
def bayes(tupla):

  email= tupla[0]
  classificacao= tupla[1]
  emailpalavras = [w for w in email if w in vocab1]

  prob_spam = log(n_docs_spam / n_docs_total)+ sum([log((cont_spam[i]+1)/n_tokens_spam + n_vocab1) for i in emailpalavras])
  prob_ham = log(n_docs_ham/ n_docs_total)+ sum([log((cont_ham[i]+1)/n_tokens_ham + n_vocab1)for i in emailpalavras])

  c = classifica(prob_spam, prob_ham)
  return email, c, classificacao

In [ ]:
#8 Classificar todo o corpus de teste passando cada mensagem pela função bayes().
classificacao_bayes =[bayes(w)for w in teste_tokens]

In [ ]:
classificacao_bayes[20]

('e', 'Email não é Spam!', 'n')

In [ ]:
#9 Avaliar a performance do classificador: para cada mensagem, comparar a classificação com as etiquetas de spam ou ham e gerar uma lista com os resultados dessa avaliação em termos de Verdadeiro Positivo (VP), Verdadeiro Negativo (VN), Falso Positivo (FP) e Falso Negativo (FN).
def Avaliacao(tupla):

    lst_cla = list()

    cl_bayes = tupla[1]
    cl_teste = tupla[2]

    if cl_bayes == 'Email não é Spam!' and cl_teste == 'HAM':

        lst_cla.append('VP')
    elif cl_bayes == 'Email não é Spam!' and cl_teste == 'SPAM':

        lst_cla.append('FP')

    elif cl_bayes == 'Email é Spam!' and cla_teste == 'SPAM':

        lst_cla.append('VN')

    else:
        lst_cla.append('FN')


    return lst_cla


In [ ]:
aval_classifica = [Avaliacao(w) for w in classificacao_bayes]
aval_class = achatar(aval_classifica)